In [8]:
#Description:
#This script is used to extract data from a website and add it to a Google spreadsheet

#the updated spreadsheet: 
#https://docs.google.com/spreadsheets/d/17LbqAvNg-QmmeQJzufPg4vqMDH4v18eK3NMe4fr4ZB8/edit#gid=0

#the website I scraped the data from:
#https://www.google.com/finance?q=tsla

#the data I extracted:
# Dow Jones Industrial Average
# S&P 500
# VIX
# Nasdaq Composite
# Tesla Inc
# Apple Inc

#Author: RichardrahciR
#Date: 2024-06-04

#libraries
#libraries versions:
#pandas==2.2.2
#numpy==1.21.2
#google-auth==2.28.0
#google-auth-httplib2==0.2.0
#google-auth-oauthlib==1.2.0
#google-api-python-client==2.132.0
#oauthlib==3.2.2
#requests==2.31.0
#beautifulsoup4==4.12.2
#lxml==4.9.3
#json
#datetime==5.5
#twilio==9.1.1


# Importing the required libraries
import pandas as pd
import numpy as np
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
import io
from googleapiclient.errors import HttpError
from datetime import datetime
from bs4 import BeautifulSoup
import requests, lxml, json
from twilio.rest import Client
import functions_framework

# Define the parameters
url = "https://www.google.com/finance?q=tsla"
#Proxy settings
params = { "hl": "en" }
headers = {
          "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
          }
#Google spreadsheet settings and credentials
file_id = "add your file id here"
scope = "add your google sheet scope here"
service_account_json_key = "add your credential file here" #credentials file
#Pointer to whether the data is updated or not
updated_or_not = False


#Function to get the data from the website
def get_ticker_data(url="https://www.google.com/finance?q=tsla",params=params,headers=headers):    
# Make a request to the website
  html = requests.get(f"{url}", params=params, headers=headers, timeout=30)
  print("request status: ", html.status_code)
#Parse the HTML content
  soup = BeautifulSoup(html.text, "lxml")

  ticker_data = []
# Extract the data from the website
  for ticker in soup.select('.tOzDHb'):
    title = ticker.select_one('.RwFyvf').text
    price = ticker.select_one('.YMlKec').text
    index = ticker.select_one('.COaKTb').text
    price_change = ticker.select_one("[jsname=Fe7oBc]")["aria-label"]

    ticker_data.append({
      "index": index,
    "title" : title,
    "price" : price,
    "price_change" : price_change
    })  
  #print if the data is empty
  if not ticker_data:
    print("Data is empty")

  result = json.dumps(ticker_data, indent=2)
  ticker_data = pd.read_json(result)
  ticker_data.drop_duplicates(subset=['title'], keep='first', inplace=True)
  print("Data collected successfully at ", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
  return ticker_data

def extract_new_data(df):
# Extract the data from the DataFrame
# data I want to extract:
# Dow Jones Industrial Average
# S&P 500
# VIX
# Nasdaq Composite
# Tesla Inc
# Apple Inc

####################################################
#This part could be edited to extract more data from the DataFrame

    dj = df[df['title'] == 'Dow Jones Industrial Average'][['price','price_change']].values
    sp = df[df['title'] == 'S&P 500'][['price','price_change']].values
    vix = df[df['title'] == 'VIX'][['price','price_change']].values
    nasdaq = df[df['title'] == 'Nasdaq Composite'][['price','price_change']].values
    tesla = df[df['title'] == 'Tesla Inc'][['price','price_change']].values
    apple = df[df['title'] == 'Apple Inc'][['price','price_change']].values
    nowtime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    added_data = [nowtime,dj[0][0],dj[0][1],sp[0][0],sp[0][1],vix[0][0],vix[0][1],nasdaq[0][0],nasdaq[0][1],tesla[0][0],tesla[0][1],apple[0][0],apple[0][1]]
    print("New data extracted successfully")
    return added_data

#Function to test if the data already exists in the DataFrame
def test_add(old_data,added_data):
    if old_data.iloc[-1].tolist() == added_data:
        new_data = old_data
        print("Data already exists, no new data added")
    else:
        # Append new data to the DataFrame
        old_data.loc[len(old_data)] = added_data
        new_data = old_data
        global updated_or_not
        updated_or_not = True
        print("New data added successfully")
    return new_data


def send_sms(body):
    ACCOUNT_SID = 'add your account sid here'
    AUTH_TOKEN = 'add your auth token here'
    TWILIO_PHONE_NUMBER = 'add your twilio phone number here'
    TO_PHONE_NUMBER = 'add your phone number here'

    client = Client(ACCOUNT_SID, AUTH_TOKEN)

    try:
        message = client.messages.create(
            body=body,
            from_=TWILIO_PHONE_NUMBER,
            to=TO_PHONE_NUMBER
        )
        print("SMS sent successfully")
    except Exception as e:
        print(f"Failed to send SMS: {e}")


def reply_to_request(request):
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
    request_json = request.get_json(silent=True)
    request_args = request.args

    if request_json and 'name' in request_json:
        name = request_json['name']
    elif request_args and 'name' in request_args:
        name = request_args['name']
    else:
        name = 'User' + str(np.random.randint(1000,9999))
    return 'Hi {}! Financial Information has been Updated.\n Richarfin, enpower your businesses'.format(name)






In [9]:
#Function to update the google sheet
@functions_framework.http
def update_google_sheet(request,spreadsheet_id,scope,service_account_json_key,sheet_name='Sheet1'):
    # Authenticate the service account
    credentials = service_account.Credentials.from_service_account_file(
                                filename=service_account_json_key, 
                                scopes=scope)
    
    # Build the service
    sheet_service = build('sheets', 'v4', credentials=credentials)
    # Get the spreadsheet with id from the sheet and convert it to a DataFrame
    sheet = sheet_service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id, range=sheet_name).execute()
    # Extract the values from the sheet
    values = sheet.get('values', [])

    # Convert to pandas DataFrame for easier manipulation
    old_google_sheet = pd.DataFrame(values[1:], columns=values[0])
    
    # Extract the new data from the website google finance
    added_data = extract_new_data(get_ticker_data())

    # Test if the data already exists in the DataFrame and add it if it doesn't
    new_google_sheet = test_add(old_google_sheet,added_data=added_data)

    
    # Convert the DataFrame to a list of lists to update the sheet
    data = [new_google_sheet.columns.to_list()] + new_google_sheet.values.tolist()
    # Update the sheet with the modified data
    sheet_service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id, range=sheet_name,
        valueInputOption='RAW', body={'values': data}).execute()
    print("Row added successfully")

    if updated_or_not:
    #put the added data in the body of the message
        body = f'''New data added successfully at {added_data[0]}: \n
    Dow Jones Industrial Average: {added_data[1]} {added_data[2]},\n
    S&P 500: {added_data[3]} {added_data[4]},\n
    VIX: {added_data[5]} {added_data[6]},\n 
    Nasdaq Composite: {added_data[7]} {added_data[8]},\n 
    Tesla Inc: {added_data[9]} {added_data[10]},\n 
    Apple Inc: {added_data[11]} {added_data[12]}\ng
    Have a nice day! Merry Christmas!'''
    send_sms(body)
    reply_message = reply_to_request(request)
    
    return reply_message + "\n" + body

        

In [10]:
added_data = update_google_sheet(file_id,scope,service_account_json_key,sheet_name='Sheet1')
print(added_data)

request status:  200
Data collected successfully at  2024-06-11 11:59:30
New data extracted successfully
New data added successfully


/var/folders/_b/4lj1bh0s4473m5xx061k14lc0000gn/T/ipykernel_33153/3957333713.py:93: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ticker_data = pd.read_json(result)


Row added successfully
SMS sent successfully
['2024-06-11 11:59:30', '38,724.85', 'Down by 0.37%', '5,366.52', 'Up by 0.11%', '12.87', 'Up by 1.02%', '17,299.45', 'Up by 0.62%', '$169.32', 'Down by 2.57%', '$205.57', 'Up by 6.45%']
